In [ ]:
# https://ibkrcampus.com/campus/ibkr-api-page/twsapi-doc/#api-introduction
#pip install requests
# https://stackoverflow.com/questions/8548030/why-does-pip-install-inside-python-raise-a-syntaxerror

In [1]:
from ib_insync import *
import requests
import pandas as pd
from bs4 import BeautifulSoup

util.startLoop()  # only use in interactive environments (i.e. Jupyter Notebooks)
# https://algotrading101.com/learn/ib_insync-interactive-brokers-api-guide/

In [2]:
#WORKING
ib = IB()
ib.connect(host='127.0.0.1', port=7496, clientId=0)
# https://www.elitetrader.com/et/threads/cant-connect-to-api-interactive-brokers.361205/
# needed to change socket port to 7496 and uncheck the Read-Only API box

<IB connected to 127.0.0.1:7496 clientId=0>

In [3]:
ib.positions()

[Position(account='U12848716', contract=Stock(conId=700941285, symbol='NUVOQ', exchange='PINK', currency='USD', localSymbol='NUVOQ', tradingClass='NUVOQ'), position=275.0, avgCost=0.24124875)]

In [4]:
sub = ScannerSubscription(
    instrument='STK', 
    locationCode='STK.NASDAQ',        # 'STK.US.MAJOR', 
    scanCode='TOP_PERC_GAIN')

tagValues = [
    TagValue("changePercAbove", "30"),
    TagValue('priceBelow', 10)]

# if I run this after-hours, it works with after-hours percentage gains!!

# the tagValues are given as 3rd argument; the 2nd argument must always be an empty list
# (IB has not documented the 2nd argument and it's not clear what it does)
scanData = ib.reqScannerData(sub, [], tagValues)

symbols = [sd.contractDetails.contract.symbol for sd in scanData]
print(symbols)

['NVNI', 'APTO', 'AADI', 'XLO']


Error 162, reqId 3: Historical Market Data Service error message:API scanner subscription cancelled: 3


In [14]:
print(symbols[0])

NVNI


In [11]:
for symbol in symbols:
    
    contract = Stock(symbol, 'SMART', 'USD')
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1500 S',   # at the end of the day would start me at 3:35 for bars[0] so effectively 25 mins
        barSizeSetting='5 mins',
        whatToShow='TRADES',
        useRTH=True,            # only grab data from Regular Trading Hours
        formatDate=1)

In [18]:
print(bars)

[BarData(date=datetime.datetime(2024, 12, 19, 15, 35, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.891, high=0.9, low=0.891, close=0.9, volume=300.0, average=0.894, barCount=3), BarData(date=datetime.datetime(2024, 12, 19, 15, 40, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.9001, high=0.905, low=0.9001, close=0.905, volume=600.0, average=0.90353, barCount=6), BarData(date=datetime.datetime(2024, 12, 19, 15, 45, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.905, high=0.905, low=0.905, close=0.905, volume=0.0, average=0.905, barCount=0), BarData(date=datetime.datetime(2024, 12, 19, 15, 50, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.9101, high=0.9101, low=0.91, close=0.9101, volume=1400.0, average=0.91003, barCount=3), BarData(date=datetime.datetime(2024, 12, 19, 15, 55, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.92, high=0.93, low=0.9059, close=0.9239, volume=6452.0, average=0.9234, barCount=16)]


In [20]:
df = pd.DataFrame(bars)

In [21]:
print(df)

                       date    open    high     low   close  volume  average  \
0 2024-12-19 15:35:00-05:00  0.8910  0.9000  0.8910  0.9000   300.0  0.89400   
1 2024-12-19 15:40:00-05:00  0.9001  0.9050  0.9001  0.9050   600.0  0.90353   
2 2024-12-19 15:45:00-05:00  0.9050  0.9050  0.9050  0.9050     0.0  0.90500   
3 2024-12-19 15:50:00-05:00  0.9101  0.9101  0.9100  0.9101  1400.0  0.91003   
4 2024-12-19 15:55:00-05:00  0.9200  0.9300  0.9059  0.9239  6452.0  0.92340   

   barCount  
0         3  
1         6  
2         0  
3         3  
4        16  


In [25]:
second_last_row_close = df.iloc[-2]
print(second_last_row_close["close"])

0.9101


In [ ]:
# track changing tickers every 30 seconds and update values
ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

In [ ]:
pending_trades = ib.reqOpenOrders() # requires that I use ClientId=0 rather than ClientId=1
# https://www.reddit.com/r/interactivebrokers/comments/119begz/cant_see_open_orders_via_api/
pending_trades

In [ ]:
symbols = [trade.contract.symbol for trade in pending_trades]
print(symbols)

In [ ]:
handling_code = [trade.order.lmtPrice for trade in pending_trades]
print(handling_code)

In [ ]:
# https://github.com/Siloaman/ib_async/blob/main/notebooks/bar_data.ipynb
# To get the earliest date of available bar data the "head timestamp" can be requested:
contract = Stock('TSLA', 'SMART', 'USD')
ib.reqHeadTimeStamp(contract, whatToShow='TRADES', useRTH=True)

In [ ]:
# Legal ones are: 
# 1 secs, 5 secs, 10 secs, 15 secs, 30 secs, 
# 1 min, 2 mins, 3 mins, 5 mins, 10 mins, 15 mins, 20 mins, 30 mins, 
# 1 hour, 2 hours, 3 hours, 4 hours, 8 hours, 
# 1 day, 1W, 1M
# To request hourly data of the last 1 day per 5 minutes:
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting='5 mins',
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1)

bars[0]

In [ ]:
df = util.df(bars)
display(df.tail(8))

In [ ]:
%matplotlib inline

In [ ]:
# There is also a utility function to plot bars as a candlestick plot. 
# It can accept either a DataFrame or a list of bars.
# Here it will print the last 10 bars:
util.barplot(bars[-10:], title=contract.symbol);

In [ ]:
# A new feature of the API is to get live updates for historical bars.
# This is done by setting endDateTime to an empty string and the keepUpToDate parameter to True.
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='480 S',          # 5 mins x 8 bars (60 x 8)
        barSizeSetting='5 mins',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=1,
        keepUpToDate=True)

In [ ]:
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

def onBarUpdate(bars, hasNewBar):
    plt.close()
    plot = util.barplot(bars)
    clear_output(wait=True)
    display(plot)

bars.updateEvent += onBarUpdate

ib.sleep(10)
ib.cancelHistoricalData(bars)

In [ ]:
# WORKING
acct_values = ib.accountValues()
#cash_balance = acct_values

#    [
#    AccountValue(account='U12848716', tag='AccountCode', value='U12848716', currency='', modelCode=''),
#    AccountValue(account='U12848716', tag='CashBalance', value='517.53', currency='USD', modelCode=''),
#    AccountValue(account='U12848716', tag='GrossPositionValue', value='1.58', currency='CAD', modelCode=''),
#    AccountValue(account='U12848716', tag='NetLiquidation', value='723.69', currency='CAD', modelCode=''),
#    AccountValue(account='U12848716', tag='TotalCashBalance', value='2.24', currency='CAD', modelCode=''),
#    AccountValue(account='U12848716', tag='TotalCashBalance', value='517.53', currency='USD', modelCode=''),
#    ]

Get Overall USD Cash Balance

In [ ]:
# WORKING
def get_cash_balance_usd(acct_values):
    for item in acct_values:
        # AccountValue(account='U12848716', tag='CashBalance', value='517.53', currency='USD', modelCode=''),
        if item[1] == 'CashBalance' and item[3] == 'USD':
            return float(item[2])  # Convert value to float for numerical use

# Get the CashBalance in USD
cash_balance_usd = get_cash_balance_usd(acct_values)
cash_balance_usd

# Retrieving Historical Data

In [ ]:
# for this to work, Market Data Subscriber Status needs to be updated for a subscription

nflx_contract = Stock('mlgo', 'SMART', 'USD')     # netflix ticker, 'smart' search for routing exchange, currency of ticker

ib.qualifyContracts(nflx_contract)

data = ib.reqMktData(nflx_contract)

In [ ]:
historical_data_nflx = ib.reqHistoricalData(
    nflx_contract, 
    '', 
    barSizeSetting='5 mins', 
    durationStr='1 D', 
    whatToShow='MIDPOINT', 
    useRTH=True
    )

historical_data_nflx[-6]      #  access the most recent candle, just like any other Python list, by using a negative index.
# https://www.interactivebrokers.com/campus/ibkr-api-page/market-data-subscriptions/
#historical_data_nflx[-1].open

In [ ]:
ib.run()